3. Realtime Conversation with Voice Translation: Description: Please create a machine learning model that facilitates real-time conversation between an English-speaking person and a Spanish-speaking person. The model should: Extract Spanish words from voice input and translate them into English, then read the translated word aloud. Similarly, take English voice input from the other user, translate it into Spanish, and read the translated word aloud. Guidelines: Make your own machine learning model. GUI is not mandatory for this. This task is a tough one. Don’t worry, accuracy doesn’t matter. The only thing matter is the amount of effort you have put. The evaluation will be conducted on models’ overall performance.

In [1]:
from glob import glob
import tensorflow as tf
import keras
from keras import layers
import os
from tensorflow.keras.saving import register_keras_serializable

In [2]:
@register_keras_serializable(package="TokenEmbedding")
class TokenEmbedding(layers.Layer):
    def __init__(self, num_vocab=1000, maxlen=100, num_hid=64):
        super().__init__()
        self.num_vocab = num_vocab
        self.maxlen = maxlen
        self.num_hid = num_hid
        
        self.emb = keras.layers.Embedding(num_vocab, num_hid)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        x = self.emb(x)
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions

    def get_config(self):
        config = super(TokenEmbedding, self).get_config()
        config.update({
            "num_vocab": self.num_vocab,
            "maxlen": self.maxlen,
            "num_hid": self.num_hid,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
    
    
@register_keras_serializable(package="SpeechFeatureEmbedding")
class SpeechFeatureEmbedding(layers.Layer):
    def __init__(self, num_hid=64, maxlen=100):
        super().__init__()
        self.num_hid = num_hid
        self.maxlen = maxlen
        
        self.conv1 = keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv2 = keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv3 = keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return self.conv3(x)

    def get_config(self):
        config = super(SpeechFeatureEmbedding, self).get_config()
        config.update({
            "num_hid": self.num_hid,
            "maxlen": self.maxlen,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [3]:
@register_keras_serializable(package="TransformerEncoder")
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.feed_forward_dim = feed_forward_dim
        self.rate = rate

        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super(TransformerEncoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "feed_forward_dim": self.feed_forward_dim,
            "rate": self.rate,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [4]:
@register_keras_serializable(package="TransformerDecoder")
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout_rate=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.feed_forward_dim = feed_forward_dim
        self.dropout_rate = dropout_rate

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.self_att = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.enc_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.self_dropout = layers.Dropout(0.5)
        self.enc_dropout = layers.Dropout(0.1)
        self.ffn_dropout = layers.Dropout(0.1)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
        )
        return tf.tile(mask, mult)

    def call(self, enc_out, target):
        input_shape = tf.shape(target)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = self.causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        target_att = self.self_att(target, target, attention_mask=causal_mask)
        target_norm = self.layernorm1(target + self.self_dropout(target_att))
        enc_out = self.enc_att(target_norm, enc_out)
        enc_out_norm = self.layernorm2(self.enc_dropout(enc_out) + target_norm)
        ffn_out = self.ffn(enc_out_norm)
        ffn_out_norm = self.layernorm3(enc_out_norm + self.ffn_dropout(ffn_out))
        return ffn_out_norm

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "feed_forward_dim": self.feed_forward_dim,
            "dropout_rate": self.dropout_rate,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)



In [5]:
@register_keras_serializable(package="Transformer")
class Transformer(keras.Model):
    def __init__(
        self,
        num_hid=64,
        num_head=2,
        num_feed_forward=128,
        source_maxlen=100,
        target_maxlen=100,
        num_layers_enc=4,
        num_layers_dec=1,
        num_classes=10,
    ):
        super().__init__()
        self.loss_metric = keras.metrics.Mean(name="loss")
        self.num_layers_enc = num_layers_enc
        self.num_layers_dec = num_layers_dec
        self.target_maxlen = target_maxlen
        self.num_classes = num_classes

        self.enc_input = SpeechFeatureEmbedding(num_hid=num_hid, maxlen=source_maxlen)
        self.dec_input = TokenEmbedding(
            num_vocab=num_classes, maxlen=target_maxlen, num_hid=num_hid
        )

        self.encoder = keras.Sequential(
            [self.enc_input]
            + [
                TransformerEncoder(num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ]
        )

        for i in range(num_layers_dec):
            setattr(
                self,
                f"dec_layer_{i}",
                TransformerDecoder(num_hid, num_head, num_feed_forward),
            )

        self.classifier = layers.Dense(num_classes)

    def decode(self, enc_out, target):
        y = self.dec_input(target)
        for i in range(self.num_layers_dec):
            y = getattr(self, f"dec_layer_{i}")(enc_out, y)
        return y

    def call(self, inputs):
        source = inputs[0]
        target = inputs[1]
        x = self.encoder(source)
        y = self.decode(x, target)
        return self.classifier(y)

    @property
    def metrics(self):
        return [self.loss_metric]

    def train_step(self, batch):
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        with tf.GradientTape() as tape:
            preds = self([source, dec_input])
            one_hot = tf.one_hot(dec_target, depth=self.num_classes)
            mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
            loss = model.compute_loss(None, one_hot, preds, sample_weight=mask)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def test_step(self, batch):
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        preds = self([source, dec_input])
        one_hot = tf.one_hot(dec_target, depth=self.num_classes)
        mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
        loss = model.compute_loss(None, one_hot, preds, sample_weight=mask)
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def generate(self, source, target_start_token_idx):
        bs = tf.shape(source)[0]
        enc = self.encoder(source)
        dec_input = tf.ones((bs, 1), dtype=tf.int32) * target_start_token_idx
        dec_logits = []
        for i in range(self.target_maxlen - 1):
            dec_out = self.decode(enc, dec_input)
            logits = self.classifier(dec_out)
            logits = tf.argmax(logits, axis=-1, output_type=tf.int32)
            last_logit = tf.expand_dims(logits[:, -1], axis=-1)
            dec_logits.append(last_logit)
            dec_input = tf.concat([dec_input, last_logit], axis=-1)
        return dec_input

    def get_config(self):
        config = super(Transformer, self).get_config()
        config.update({
            "num_hid": self.num_hid,
            "num_head": self.num_head,
            "num_feed_forward": self.num_feed_forward,
            "source_maxlen": self.source_maxlen,
            "target_maxlen": self.target_maxlen,
            "num_layers_enc": self.num_layers_enc,
            "num_layers_dec": self.num_layers_dec,
            "num_classes": self.num_classes,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [6]:
saveto = "./datasets/spanishdataset1/"
wavs = glob("{}/**/converted/*.wav".format(saveto), recursive=True)

In [7]:
wavs

['./datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0000.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0001.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0002.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0003.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0004.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0005.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0006.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0007.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0008.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0009.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0010.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0011.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0012.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\

In [8]:
wavs

['./datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0000.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0001.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0002.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0003.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0004.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0005.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0006.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0007.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0008.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0009.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0010.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0011.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\\19demarzo_0012.wav',
 './datasets/spanishdataset1\\19demarzo\\converted\

In [9]:
id_to_text = {}
with open("datasets/spanishdataset1/transcript.txt", encoding="utf-8") as f:
    for line in f.readlines():
        id = line.strip().split("|")[0]
        text = line.strip().split("|")[1]
        id_to_text[id] = text

In [10]:
id_to_text["19demarzo/19demarzo_0000.wav"]

'Durante nuestra conversación advertí que la multitud aumentaba, apretándose más.'

In [11]:
max([len(lines) for lines in id_to_text.values()])

382

In [12]:
for w in wavs:
    a=w.strip().split("\\")[1:]
    aa="".join(a)
    print(aa)

19demarzoconverted19demarzo_0000.wav
19demarzoconverted19demarzo_0001.wav
19demarzoconverted19demarzo_0002.wav
19demarzoconverted19demarzo_0003.wav
19demarzoconverted19demarzo_0004.wav
19demarzoconverted19demarzo_0005.wav
19demarzoconverted19demarzo_0006.wav
19demarzoconverted19demarzo_0007.wav
19demarzoconverted19demarzo_0008.wav
19demarzoconverted19demarzo_0009.wav
19demarzoconverted19demarzo_0010.wav
19demarzoconverted19demarzo_0011.wav
19demarzoconverted19demarzo_0012.wav
19demarzoconverted19demarzo_0013.wav
19demarzoconverted19demarzo_0014.wav
19demarzoconverted19demarzo_0015.wav
19demarzoconverted19demarzo_0016.wav
19demarzoconverted19demarzo_0017.wav
19demarzoconverted19demarzo_0018.wav
19demarzoconverted19demarzo_0019.wav
19demarzoconverted19demarzo_0020.wav
19demarzoconverted19demarzo_0021.wav
19demarzoconverted19demarzo_0022.wav
19demarzoconverted19demarzo_0023.wav
19demarzoconverted19demarzo_0024.wav
19demarzoconverted19demarzo_0025.wav
19demarzoconverted19demarzo_0026.wav
1

In [13]:
for w in wavs:
    l=w.strip().split("\\")[1:]
    la="".join(l)
    laa=la.split("converted")
    laaa="/".join(laa)
    print(laaa)

19demarzo/19demarzo_0000.wav
19demarzo/19demarzo_0001.wav
19demarzo/19demarzo_0002.wav
19demarzo/19demarzo_0003.wav
19demarzo/19demarzo_0004.wav
19demarzo/19demarzo_0005.wav
19demarzo/19demarzo_0006.wav
19demarzo/19demarzo_0007.wav
19demarzo/19demarzo_0008.wav
19demarzo/19demarzo_0009.wav
19demarzo/19demarzo_0010.wav
19demarzo/19demarzo_0011.wav
19demarzo/19demarzo_0012.wav
19demarzo/19demarzo_0013.wav
19demarzo/19demarzo_0014.wav
19demarzo/19demarzo_0015.wav
19demarzo/19demarzo_0016.wav
19demarzo/19demarzo_0017.wav
19demarzo/19demarzo_0018.wav
19demarzo/19demarzo_0019.wav
19demarzo/19demarzo_0020.wav
19demarzo/19demarzo_0021.wav
19demarzo/19demarzo_0022.wav
19demarzo/19demarzo_0023.wav
19demarzo/19demarzo_0024.wav
19demarzo/19demarzo_0025.wav
19demarzo/19demarzo_0026.wav
19demarzo/19demarzo_0027.wav
19demarzo/19demarzo_0028.wav
19demarzo/19demarzo_0029.wav
19demarzo/19demarzo_0030.wav
19demarzo/19demarzo_0031.wav
19demarzo/19demarzo_0032.wav
19demarzo/19demarzo_0033.wav
19demarzo/19de

In [14]:
def get_data(wavs, id_to_text, maxlen=50):
    """returns mapping of audio paths and transcription texts"""
    data = []
    for w in wavs:
        splitted_w=w.strip().split("\\")[1:]
        joined_w="".join(splitted_w)
        splitted_joined_w=joined_w.split("converted")
        id="/".join(splitted_joined_w)
        if len(id_to_text[id]) < maxlen:
            data.append({"audio": w, "text": id_to_text[id]})
    return data

In [15]:
@register_keras_serializable(package="VectorizeChar")
class VectorizeChar:
    def __init__(self, max_len=50):
        self.vocab = (
            ["-", "#", "<", ">"]
            + [chr(i + 96) for i in range(1, 27)]
            + [" ", ".", ",", "?"]
        )
        self.max_len = max_len
        self.char_to_idx = {}
        for i, ch in enumerate(self.vocab):
            self.char_to_idx[ch] = i

    def __call__(self, text):
        text = text.lower()
        text = text[: self.max_len - 2]
        text = "<" + text + ">"
        pad_len = self.max_len - len(text)
        return [self.char_to_idx.get(ch, 1) for ch in text] + [0] * pad_len

    def get_vocabulary(self):
        return self.vocab
    
    def get_config(self):
        return {
            "max_len": self.max_len,
            "vocab": self.vocab,
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)



max_target_len = 385 # all transcripts in out data are < 385 characters
data = get_data(wavs, id_to_text, max_target_len)
vectorizer = VectorizeChar(max_target_len)
print("vocab size", len(vectorizer.get_vocabulary()))


def create_text_ds(data):
    texts = [_["text"] for _ in data]
    text_ds = [vectorizer(t) for t in texts]
    text_ds = tf.data.Dataset.from_tensor_slices(text_ds)
    return text_ds


def path_to_audio(path):
    # spectrogram using stft
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1)
    audio = tf.squeeze(audio, axis=-1)
    stfts = tf.signal.stft(audio, frame_length=200, frame_step=80, fft_length=256)
    x = tf.math.pow(tf.abs(stfts), 0.5)
    # normalisation
    means = tf.math.reduce_mean(x, 1, keepdims=True)
    stddevs = tf.math.reduce_std(x, 1, keepdims=True)
    x = (x - means) / stddevs
    audio_len = tf.shape(x)[0]
    # padding to 10 seconds
    pad_len = 2754
    paddings = tf.constant([[0, pad_len], [0, 0]])
    x = tf.pad(x, paddings, "CONSTANT")[:pad_len, :]
    return x


def create_audio_ds(data):
    flist = [_["audio"] for _ in data]
    audio_ds = tf.data.Dataset.from_tensor_slices(flist)
    audio_ds = audio_ds.map(path_to_audio, num_parallel_calls=tf.data.AUTOTUNE)
    return audio_ds


def create_tf_dataset(data, bs=4):
    audio_ds = create_audio_ds(data)
    text_ds = create_text_ds(data)
    ds = tf.data.Dataset.zip((audio_ds, text_ds))
    ds = ds.map(lambda x, y: {"source": x, "target": y})
    ds = ds.batch(bs)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


split = int(len(data) * 0.99)
train_data = data[:split]
test_data = data[split:]
ds = create_tf_dataset(train_data, bs=64)
val_ds = create_tf_dataset(test_data, bs=4)

vocab size 34


In [16]:
id_to_text

{'19demarzo/19demarzo_0000.wav': 'Durante nuestra conversación advertí que la multitud aumentaba, apretándose más.',
 '19demarzo/19demarzo_0001.wav': 'Componíanla personas de ambos sexos y de todas las clases de la sociedad,',
 '19demarzo/19demarzo_0002.wav': 'espontáneamente venidas por uno de esos llamamientos morales, íntimos, misteriosos, informulados, que no parten de ninguna voz oficial,',
 '19demarzo/19demarzo_0003.wav': 'y resuenan de improviso en los oídos de un pueblo entero, hablándole el balbuciente lenguaje de la inspiración.',
 '19demarzo/19demarzo_0004.wav': 'La campana de ese arrebato glorioso no suena sino cuando son muchos los corazones dispuestos a palpitar en concordancia con su anhelante ritmo,',
 '19demarzo/19demarzo_0005.wav': 'y raras veces presenta la historia ejemplos como aquel, porque el sentimiento patrio no hace milagros sino cuando es una condensación colosal,',
 '19demarzo/19demarzo_0006.wav': 'una unidad sin discrepancias de ningún género,',
 '19demarzo

In [17]:
@register_keras_serializable(package="DisplayOutputs")
class DisplayOutputs(keras.callbacks.Callback):
    def __init__(
        self, batch, idx_to_token, target_start_token_idx=27, target_end_token_idx=28
    ):
        """Displays a batch of outputs after every epoch

        Args:
            batch: A test batch containing the keys "source" and "target"
            idx_to_token: A List containing the vocabulary tokens corresponding to their indices
            target_start_token_idx: A start token index in the target vocabulary
            target_end_token_idx: An end token index in the target vocabulary
        """
        self.batch = batch
        self.target_start_token_idx = target_start_token_idx
        self.target_end_token_idx = target_end_token_idx
        self.idx_to_char = idx_to_token

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 != 0:
            return
        source = self.batch["source"]
        target = self.batch["target"].numpy()
        bs = tf.shape(source)[0]
        preds = self.model.generate(source, self.target_start_token_idx)
        preds = preds.numpy()
        for i in range(bs):
            target_text = "".join([self.idx_to_char[_] for _ in target[i, :]])
            prediction = ""
            for idx in preds[i, :]:
                prediction += self.idx_to_char[idx]
                if idx == self.target_end_token_idx:
                    break
            print(f"target:     {target_text.replace('-','')}")
            print(f"prediction: {prediction}\n")
    def get_config(self):
        return {
            "batch": self.batch,
            "idx_to_token": self.idx_to_char,
            "target_start_token_idx": self.target_start_token_idx,
            "target_end_token_idx": self.target_end_token_idx,
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [18]:
@register_keras_serializable(package="CustomSchedule")
class CustomSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(
        self,
        init_lr=0.00001,
        lr_after_warmup=0.001,
        final_lr=0.00001,
        warmup_epochs=15,
        decay_epochs=85,
        steps_per_epoch=203,
    ):
        super().__init__()
        self.init_lr = init_lr
        self.lr_after_warmup = lr_after_warmup
        self.final_lr = final_lr
        self.warmup_epochs = warmup_epochs
        self.decay_epochs = decay_epochs
        self.steps_per_epoch = steps_per_epoch

    def calculate_lr(self, epoch):
        """linear warm up - linear decay"""
        warmup_lr = (
            self.init_lr
            + ((self.lr_after_warmup - self.init_lr) / (self.warmup_epochs - 1)) * epoch
        )
        decay_lr = tf.math.maximum(
            self.final_lr,
            self.lr_after_warmup
            - (epoch - self.warmup_epochs)
            * (self.lr_after_warmup - self.final_lr)
            / self.decay_epochs,
        )
        return tf.math.minimum(warmup_lr, decay_lr)

    def __call__(self, step):
        epoch = step // self.steps_per_epoch
        epoch = tf.cast(epoch, "float32")
        return self.calculate_lr(epoch)
    
    def get_config(self):
        return {
            'init_lr': self.init_lr,
            'lr_after_warmup': self.lr_after_warmup,
            'final_lr': self.final_lr,
            'warmup_epochs': self.warmup_epochs,
            'decay_epochs': self.decay_epochs,
            'steps_per_epoch': self.steps_per_epoch
        }

In [19]:
batch = next(iter(val_ds))
                                
# The vocabulary to convert predicted indices into characters
idx_to_char = vectorizer.get_vocabulary()
display_cb = DisplayOutputs(
    batch, idx_to_char, target_start_token_idx=2, target_end_token_idx=3
)  # set the arguments as per vocabulary index for '<' and '>'

model = Transformer(
    num_hid=200,
    num_head=2,
    num_feed_forward=400,
    target_maxlen=max_target_len,
    num_layers_enc=4,
    num_layers_dec=1,
    num_classes=34,
)
loss_fn = keras.losses.CategoricalCrossentropy(
    from_logits=True,
    label_smoothing=0.1,
)
learning_rate = CustomSchedule(
    init_lr=0.00001,
    lr_after_warmup=0.001,
    final_lr=0.00001,
    warmup_epochs=15,
    decay_epochs=85,
    steps_per_epoch=len(ds),
)
optimizer = keras.optimizers.Adam(learning_rate)
model.compile(optimizer=optimizer, loss=loss_fn,metrics=["accuracy"])
history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb], epochs=20)

Epoch 1/20

172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - loss: 1.1094

c:\python312\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (4, 2, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


target:     <grit# in#s abraz#ndole, llenos los ojos de l#grimas. la condesa no dec#a nada y lloraba tambi#n. santorcaz,>
prediction: <en des la visa de la des la eses pe la do a ena a eno e s la do co de a es me s es es a la da la sas e la do e de den do s e des la des an de en ca l pe tora e e e a de pa la la. ca des a de da as ee da dos l en, l la a de ano dento l l pe dos e e dos do e es stla des enos s l an dan s an an l l a e a s s a der mes de denos a doracia rdo s des do posa de l ta da e s a do es pe la e

target:     <despu#s de aquella crisis de su esp#ritu, cay# en nuevo sopor profund#simo,y cerca de la madrugada, recobr# el conocimiento con un despertar sereno y sosegado.>
prediction: <en des la e sa de la des la eses e ela do a ena a eno e s la do co de a es me s es es a la da la sas e la do e de den do s e des la des an de en ca l pe tora e e e a de pa la la. ca des a de da as ee da dos l en, l la a de ano dento l l pe dos e e dos do e es stla des enos s l an dan s an an

In [20]:
model.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ speech_feature_embedding        │ (None, None, 200)      │     1,164,400 │
│ (SpeechFeatureEmbedding)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_embedding                 │ ?                      │        83,800 │
│ (TokenEmbedding)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (None, None, 200)      │     3,095,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder             │ ?                      │       804,600 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 384, 34)        │         6,834 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,972,503 (45.67 MB)

 Trainable params: 3,990,834 (15.22 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,981,669 (30.45 MB)

In [21]:
model.save("spa_speech_recognition.keras")

AttributeError: 'Transformer' object has no attribute 'num_hid'

In [ ]:
model.save_weights("spa_speech_recognition.weights.h5")

: 

In [ ]:
def preprocess_audio(path):
    # Preprocess the audio file similar to training
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1)
    audio = tf.squeeze(audio, axis=-1)
    stfts = tf.signal.stft(audio, frame_length=200, frame_step=80, fft_length=256)
    x = tf.math.pow(tf.abs(stfts), 0.5)
    means = tf.math.reduce_mean(x, 1, keepdims=True)
    stddevs = tf.math.reduce_std(x, 1, keepdims=True)
    x = (x - means) / stddevs
    pad_len = 2754
    paddings = tf.constant([[0, pad_len], [0, 0]])
    x = tf.pad(x, paddings, "CONSTANT")[:pad_len, :]
    return x


: 

In [ ]:
preprocessed_audio=preprocess_audio("batalla_arapiles_4595.wav")
# Prepare the input for the model
preprocessed_audio = tf.expand_dims(preprocessed_audio, axis=0)  # Add batch dimension
target_start_token_idx = 2  # Update this based on your vocabulary
# Generate predictions
predictions = model.generate(preprocessed_audio, target_start_token_idx)
decoded_predictions = predictions.numpy()
# Decode the predictions using idx_to_char
predicted_text = ''.join([idx_to_char[idx] for idx in decoded_predictions[0]])
print("Predicted text:", predicted_text)

: 

In [ ]:
import json
with open("spa_speech_id_to_char",'w',encoding="utf-8") as f:
    json.dump(idx_to_char,f)

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 